اول حاجه هنحط الداتا في مونجو دي بي 

In [ ]:
#this is new DAG
import json
from pymongo import MongoClient

#  الاتصال بـ MongoDB
client = MongoClient("mongodb://localhost:27017/")

#  اختيار Database و Collection
db = client["nyc_data"]
collection = db["nyc_traffic"]

# (اختياري) تفريغ الكوليكشن لو بتجرب
# collection.delete_many({})

#  قراءة ملف الـ JSON
with open("nyc_traffic.json", "r") as f:
    data = json.load(f)

# تأكيد إن الداتا Array
assert isinstance(data, list), "JSON file must contain a list of documents"

#  إدخال الدhتا
collection.insert_many(data)

print(f" Inserted {len(data)} documents into MongoDB successfully")


✅ Inserted 41362 documents into MongoDB successfully


حولنا الداتا بتتاعت الشوارع من ملف سي اي في اي  ملف  parquet
ودي عشان ننوع من مصارد الداتا الي جايلنا 

In [1]:

import pandas as pd 
data = pd.read_csv("Centerline.csv")
data.to_parquet("streets_data.parquet")

انشاء اول تاسك في الداج وده هنعمل كونكت مع الاس 3 و هنكريت باكت 
ونعمل فيها 2 فولدر وبعدين نرفع الرو داتا فيها 
1- هنجيب الداتا  من مونجو دي بي ونرفعها علي هئيه جيسون فايل 
2- هنجيب الفايل البركيه ونرفعه 
3- هنجيب الفايل الثالث والاخير ونرفعه الي هو csv 

In [ ]:
#task 1
from pymongo import MongoClient
import json
import boto3

# اتصال Mongo
client = MongoClient("mongodb://localhost:27017")
db = client["nyc_data"]
collection = db["nyc_traffic"]

#-----------------------------------------------
# connect with s3 
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)
Bucket="datalake"

#------------------------------------------------
# create bucket لو مش موجود 
try:
     s3.create_bucket(Bucket="datalake")
except:
     pass

#------------------------------------------------
# create 2 folder (raw , processed)
try:
     s3.put_object(Bucket="datalake", Key="raw/")
     s3.put_object(Bucket="datalake", Key="processed/")
except:
     pass

#----------------------------------------------------------
# upload first file to s3 (json file)
cursor = collection.find({}, {"_id": 0})

data = list(cursor)          # in-memory
json_bytes = json.dumps(
    data,
    ensure_ascii=False
).encode("utf-8")

#load to s3
s3.Bucket("datalake").put_object(
    Key="raw/mongo_traffic.json",
    Body=json_bytes,
    ContentType="application/json"
)


#---------------------------------------------------==
# upload second file csv to s3
s3_client = boto3.client(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)


s3_client.upload_file(
    Filename="Motor_Vehicle_Collisions_Crashes.csv",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/Motor_Vehicle_Collisions_Crashes.csv",    # المسار داخل الباكيت
    ExtraArgs={"ContentType": "text/csv"}   # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)

#--------------------------------------
# upload the last file (parquet file )

s3_client.upload_file(
    Filename="streets_data.parquet",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/streets_data.parquet" ,   # المسار داخل الباكيت
    ExtraArgs={"ContentType": "application/octet-stream"}  # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)




KeyboardInterrupt: 

26/01/15 19:32:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 55852438 ms exceeds timeout 120000 ms
26/01/15 19:32:34 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/15 19:32:35 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1324)
	at

التاسك التاني هنجيب الداتا بتاعت الجيسون فايل ونحولها لداتا فريم 
دا بستخدام باي سبارك و نعمل عليها بروسيسنج وبعدين نرفعها علي س3 تاني بس المره ده في فولدر اسمه بروسيس داتا 


In [13]:
# task 2
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , lower ,to_date
spark = (
    SparkSession.builder
    .appName("transform1")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

#spark.sparkContext.setLogLevel("WARN")
# هنجيب الداتا بقا ونحولها لداتا فريم

df = spark.read.json(
    "s3a://datalake/raw/mongo_traffic.json"
)

#--------------------------------------------------------
# تعديل القيم في الاعمده لتصبح كلها سمول 

cols_to_lower = ["borough", "incident_type", "severity", "status", "street_name", "weather_condition"]

for c in cols_to_lower:
    df = df.withColumn(c, lower(col(c)))

#--------------------------------------------------------
# تغير الداتا تايب بتاع الديت 
df1 = df.withColumn(
    "created_date", to_date(col("created_date"), "yyyy-MM-dd"))

# تغير اسماء الاعمده 
rename_coulmn={
    "fatalities":"pepole_killed",
    "incident_type":"complaint_type",
    "incident_id":"complaint_311_id",
    "injuries_reported":"pepole_injuries",
}

for  old,new in rename_coulmn.items():
    df1=df1.withColumnRenamed(old,new)

#------------------------------------
df1.write \
    .mode("overwrite") \
    .option("header","true") \
    .csv("s3a://datalake/processed/complaint_311")

df1.show(10)



26/01/12 07:30:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
26/01/12 07:30:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/12 07:30:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/12 07:30:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+---------+------------+-------------+--------------------+-------------------+---------------+---------+----------+---------------------+--------+------+---------------+--------------------+-----------------+-----------------+
|  borough|created_date|pepole_killed|    complaint_311_id|     complaint_type|pepole_injuries| latitude| longitude|response_time_minutes|severity|status|    street_name|traffic_volume_level|vehicles_involved|weather_condition|
+---------+------------+-------------+--------------------+-------------------+---------------+---------+----------+---------------------+--------+------+---------------+--------------------+-----------------+-----------------+
|   queens|  2025-01-01|            0|99a27f0e-637a-45d...| signal malfunction|              0|40.758144|-73.800187|                   17|  medium|  open|     2nd avenue|               Light|                3|            clear|
| brooklyn|  2025-01-01|            0|8424a97b-e059-4e6...|    illegal parking|         

التاسك التاني 
ودي عباره عن اننا هنجيب الفايل الي هو csv 
وهنبدء نتعامل معاه علي pyspark 
 بحيث اننا نعمل بروسيسنج لداتا الي فيه بما يتناسب مع احتياجتنا 
 وفي الاخر نرفع الفايل علي هئيه بركيه فايل علي s3
جوا الفولدر الي اسمه processed

In [8]:
# task 3
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp , lower



spark = (
    SparkSession.builder
    .appName("transform2")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Motor_Vehicle_Collisions_Crashes.csv",
    inferSchema=True,
    header=True
)



#----------------------------------------------------------
# معالجه الداتا الي في الفايل 
#----------------------------------------------------------



# تحدبد الاعمده الي محتجنيها بس 
df_selected=df.select("COLLISION_ID","CRASH DATE","CRASH TIME","BOROUGH","VEHICLE TYPE CODE 1"
              ,"ON STREET NAME","NUMBER OF PERSONS INJURED","NUMBER OF PERSONS KILLED"
              , "CONTRIBUTING FACTOR VEHICLE 1") 

#----------------------------------
# تغير الداتا تيب بتاع عمود التاريخ والقيم الي مش هتفع تبقا تاريخ تيبقا نل
df1 = df_selected.withColumn(
    "CRASH DATE",
    expr("try_to_date(`CRASH DATE`, 'M/d/yyyy')")
)

#----------------------------------
#  في عمود الديت هنعمل فلتره بحيث نجيب  القيم الي مش بنل 
df2 = df1.filter(col("CRASH DATE").isNotNull())

#----------------------------------
# فلتره الداتا لعام 2025 بس 
df_2025_plus = df2.filter(
    col("CRASH DATE") > "2025-01-01"
)


#----------------------------------
# هنحاول اننا نعدل القيم الفاضيه بنل دي بعدين ان كان في وقت 
# بس دلوقتي هنحط  القيم الفاضيه دي unknown
df_fill = df_2025_plus.fillna({
    "BOROUGH": "UNKNOWN",          # أي Null في BOROUGH هيتحول لـ "UNKNOWN"
    "ON STREET NAME": "UNKNOWN STREET" , # أي Null في ON STREET NAME هيتحول لـ "UNKNOWN STREET"
    "NUMBER OF PERSONS INJURED":"0", # عدد الاصابات الي بنل هحط مكانه 0
    "VEHICLE TYPE CODE 1":"Sedan" # عندي 1417 قيمه  بنل هعوضهم  ب اكتر قيمه اتكرت 
    
})

#----------------------------------------------------------
# تحويل الداتا تايب بتاع العمود الاصابات الي  int 
df_4 = df_fill.withColumn(
    "NUMBER OF PERSONS INJURED",
    col("NUMBER OF PERSONS INJURED").cast("int")
)
from pyspark.sql.functions import to_timestamp , date_format
#----------------------------------
# تحويل الداتا الي في العمود بتاع الوقت لصيغه ديت
df_time = df_4.withColumn(
    "CRASH TIME",
    to_timestamp(col("CRASH TIME"), "H:mm")
)

#----------------------------------
# HH:MM تحديد ان الديت يبقا بصيغه الساعه والدقائق فقط  
df_final = df_time.withColumn(
    "CRASH TIME",
    date_format("CRASH TIME", "HH:MM")
)

#=========================================
# تغير اسماء الاعمده
rename_dict = {
    "CRASH DATE": "crash_date",
    "CRASH TIME": "crash_time",
    "ON STREET NAME": "street_name",
    "NUMBER OF PERSONS INJURED": "persons_injured",
    "NUMBER OF PERSONS KILLED": "persons_killed",
    "VEHICLE TYPE CODE 1": "vehicle_type",
    "CONTRIBUTING FACTOR VEHICLE 1": "contributing_factor",
    "COLLISION_ID": "collision_id",
    "BOROUGH": "borough"
}

for old, new in rename_dict.items():
    df_final = df_final.withColumnRenamed(old, new)

# هنحول الداتا الي في الاعمده كلها لسمول  
df_final = df_final.withColumn("borough", lower(col("borough")))
df_final = df_final.withColumn("vehicle_type", lower(col("vehicle_type")))
df_final = df_final.withColumn("street_name", lower(col("street_name")))
df_final = df_final.withColumn("contributing_factor", lower(col("contributing_factor")))

df_final.show(10)



# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
df_final.write \
    .mode("overwrite") \
    .option("header","true") \
    .csv("s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes")




+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|collision_id|crash_date|crash_time|  borough|vehicle_type|      street_name|persons_injured|persons_killed| contributing_factor|
+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|     4803256|2025-04-02|     09:01|   queens|   ambulance|   unknown street|              0|             0|         unspecified|
|     4804525|2025-04-07|     12:01|    bronx|       sedan|bedford park blvd|              0|             0|         unspecified|
|     4804466|2025-04-07|     06:01|    bronx|       sedan|   unknown street|              0|             0|        unsafe speed|
|     4790397|2025-01-31|     08:01|   queens|       sedan|        exeter st|              1|             0|driver inattentio...|
|     4790246|2025-02-02|     20:01|  unknown|       sedan|   unknown street|             

26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:39 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:39 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:48 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


task4
------
هنجيب الداتا للفايل الثالث من s3
وهنعمل بروسيسنج علي الفايل ده برضو 
وبعدين نرفع الفايل تاني علي s3
في فولدر الداتا بروسيسد

In [10]:
# task 4
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp , lower

spark = (
    SparkSession.builder
    .appName("transform3")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Centerline.csv",
    header=True,
    inferSchema=True
)

#--------------------------------------
#هنجيب الاعمده الي محتجينها بس 
df=df.select("STREET NAME","GlobalID","Number Park Lanes","Number Travel Lanes","Snow Priority",
          "Street Width","BIKE_LANE")

#--------------------------------------
# fillna
df=df.fillna(
    {
        "Number Park Lanes":0, # تعويض الارقام الي ب نل ب 0
        "Number Travel Lanes":2,
        "Snow Priority":"UNKNOWN",
        "Street Width":30,
        "BIKE_LANE":0

    }
)
#--------------------------------------
# تغير اسماء الاعمده 
rename_dic={
    "STREET NAME":"street_name",
    "GlobalID":"global_id",
    "Number Park Lanes":"number_park_lanes",
    "Number Travel Lanes":"number_travel_lanes",
    "Snow Priority":"snow_priority",
    "Street Width":"street_width",
    "BIKE_LANE":"bike_lane"

}

for old , new in rename_dic.items():
     df=df.withColumnRenamed(old, new)
#  هنحول القيم الي في الاعمده الاسترينج الي قيم كلها سمول 
df = df.withColumn("street_name",lower(col("street_name")))


df.show(10)
#--------------------------------------
# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
df.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/processed/streets_data")

+------------------+--------------------+-----------------+-------------------+-------------+------------+---------+
|       street_name|           global_id|number_park_lanes|number_travel_lanes|snow_priority|street_width|bike_lane|
+------------------+--------------------+-----------------+-------------------+-------------+------------+---------+
|                 n|cedc2dde-7e8b-442...|                2|                  2|            C|          34|        0|
|              hone|9c163e85-23ad-418...|                2|                  2|            S|          32|        0|
|                48|fdccf94f-201f-431...|                2|                  1|            S|          32|        0|
|            laight|bcbbb800-b963-45b...|                2|                  1|            S|          34|        0|
|             brook|d7f8c5d8-637b-412...|                1|                  2|            H|          25|        0|
|                60|27fcb089-c93b-41f...|                2|     

26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


#Task 5
 هنجيب الداتا كلها ونبدء نعمل جوينز عشان نكون   الديمنشز 
 هنرفع  الديمنشنز علي s3  
 في فولدر جديد اسمه data_wharehouse(dim) 
 

In [2]:
#task 5
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from pyspark.sql.window import Window





# 1-create Dim_Borough
def Dim_Borough(df):
        dim_borough=df.select("borough").distinct()

        #  نضيف عمود ID فريد لكل بورغو
        dim_borough = dim_borough.withColumn("borough_id", monotonically_increasing_id())

        # 4- نرتب الأعمدة بحيث يكون ID الأول
        dim_borough = dim_borough.select("borough_id", "borough")

        # 5- نعرض عينة للتأكد
        #dim_borough.show(10, truncate=False)
        return(dim_borough)
    
#==================================================
#  2-create Dim_Status
def Dim_Status(df):
     dim_status=df.select("status").distinct()

     #  نضيف عمود ID فريد لكل بورغو
     dim_status = dim_status.withColumn("status_key", monotonically_increasing_id())

     # 4- نرتب الأعمدة بحيث يكون ID الأول
     dim_status = dim_status.select("status_key", "status")

    # 5- نعرض عينة للتأكد
     dim_status.show(10, truncate=False)

     return(dim_status)


#=================================================
# 3-create Dim_Traffic_Volume_Level
def Dim_Traffic_Volume_Level(df):
     dim_traffic_volume_level=df.select("traffic_volume_level").distinct()

     #  نضيف عمود ID فريد لكل بورغو
     dim_traffic_volume_level = dim_traffic_volume_level.withColumn("volume_level_key", monotonically_increasing_id())

     # - نرتب الأعمدة بحيث يكون ID الأول
     dim_traffic_volume_level = dim_traffic_volume_level.select("volume_level_key", "traffic_volume_level")

     #تغير اسماء الاعمده 
     dim_traffic_volume_level=dim_traffic_volume_level.withColumnRenamed("traffic_volume_level","volume_level")


     #  5- نعرض عينة للتأكد
     #dim_traffic_volume_level.show(10, truncate=False)
     return(dim_traffic_volume_level)


#====================================================
# 4-create Dim_Vehicle_Type 
def Dim_Vehicle_Type(df):
     dim_vehicle_type=df.select("vehicle_type").distinct()

     #  نضيف عمود ID فريد لكل سياره 
     dim_vehicle_type = dim_vehicle_type.withColumn("vehicle_type_id", monotonically_increasing_id())

     #  نرتب الأعمدة بحيث يكون ID الأول
     dim_vehicle_type = dim_vehicle_type.select("vehicle_type_id", "vehicle_type")


     #تغير اسماء الاعمده 
     dim_vehicle_type=dim_vehicle_type.withColumnRenamed("vehicle_type","vehicle_name")

     #  نعرض عينة للتأكد
     #print(dim_vehicle_type.count())
     return(dim_vehicle_type)


#============================================================
# 5-create Dim_Weather 
def Dim_Weather(df):
     dim_weather=df.select("weather_condition").distinct()

     #  نضيف عمود ID فريد لكل طقس 
     dim_weather = dim_weather.withColumn("weather_key", monotonically_increasing_id())

     #  نرتب الأعمدة بحيث يكون ID الأول
     dim_weather = dim_weather.select("weather_key", "weather_condition")

     #  نعرض عينة للتأكد
     #print(dim_weather.count())
     #dim_weather.show()
     return(dim_weather)


#============================================================
# 6-create Dim_Contributing
def Dim_Contributing(df):
     dim_contributing=df.select("contributing_factor").distinct()

     #  نضيف عمود ID فريد لكل سبب حادث 
     dim_contributing = dim_contributing.withColumn("factor_key", monotonically_increasing_id())

     #  نرتب الأعمدة بحيث يكون ID الأول
     dim_contributing = dim_contributing.select("factor_key", "contributing_factor")

     #  نعرض عينة للتأكد
     #print(dim_contributing.count())
     #dim_contributing.show()
     return(dim_contributing)

#=====================================================================

# 7-create Dim_Street 
def Dim_Street(df1,df2,df3):

     # هنحاول اننا نجمع اسماء الشوارع الي في ال 3 داتا فريم هنا 
     # مع العلم ان الشوارع الي في الداتا فريمز الي هي غير بتاعت الشارع ملهاش قيم غير اسم الشارع بنحاول اننا نعوض ده ب فيمه نن

     # في الاول هنجيب اسماء الشوارع الي الداتا فريم بتاعت الحوادث 
     streets_collisions = df1.select("street_name").distinct()

     #  وبعدين نجيب الشوارع الي داتا فريم بتاعت complaint_311
     streets_complaint_311= df2.select("street_name").distinct()

     #  وهنا هنجيب اسماء الشوارع من الداتا فريم الي خاصه بشوارع اصلا 
     streets_streets_data=df3.select(
         "street_name",
         "number_park_lanes",
         "number_travel_lanes",
         "snow_priority",
         "street_width",
         "bike_lane")


     dim_street = (
         streets_streets_data
    .unionByName(
        streets_collisions
        .withColumn("number_park_lanes", lit(None))
        .withColumn("number_travel_lanes", lit(None))
        .withColumn("street_width", lit(None))
        .withColumn("bike_lane", lit(None))
        .withColumn("snow_priority", lit(None)),
        allowMissingColumns=True
    )
              .unionByName(
               streets_complaint_311
              .withColumn("number_park_lanes", lit(None))
              .withColumn("number_travel_lanes", lit(None))
              .withColumn("street_width", lit(None))
              .withColumn("bike_lanes", lit(None))
              .withColumn("snow_priority", lit(None)),
              allowMissingColumns=True

             )
        )

     #هنمسح  الشوارع المكرره 
     dim_street =dim_street.dropDuplicates(['street_name'])

     #  نضيف عمود ID فريد لكل شارع  
     dim_street = dim_street.withColumn("street_id", monotonically_increasing_id())

     #  نرتب الأعمدة بحيث يكون ID الأول
     dim_street = dim_street.select(
         "street_id", 
         "street_name",
         "number_park_lanes",
         "number_travel_lanes",
         "snow_priority",
         "street_width",
         "bike_lane")

     # هنحاول اننا نملي القيم الفاضيه بقيم مناسبه 

     dim_street=dim_street.fillna({
         "snow_priority":"UNKNOWN",
         "street_width":30,
         "bike_lane":0,
         "number_park_lanes":1,
         "number_travel_lanes":2

         })
     #  نعرض عينة للتأكد
     #print(dim_street.count())
     #dim_street.show(10)
     return(dim_street)

#=====================================================
# 8-create Dim_Complaint
def Dim_Complaint(df):
     dim_complaint=df.select("complaint_type")
    
     #هنمسح  انواع الشكاوي  المكرره  المكرره 
     dim_complaint =dim_complaint.dropDuplicates(['complaint_type'])

     #  نضيف عمود ID فريد لكل شارع  
     dim_complaint = dim_complaint.withColumn("complaint_key", monotonically_increasing_id())

     #  نرتب الأعمدة بحيث يكون ID الأول
     dim_complaint = dim_complaint.select(
         "complaint_key", 
         "complaint_type")

     #  نعرض عينة للتأكد
     #print(dim_complaint.count())
     #dim_complaint.show(10)
     return(dim_complaint)


#=======================================================
# 9-Dim_Date 
def Dim_Date(spark):
     dim_date = spark.sql("""
         SELECT explode(
             sequence(
                 to_date('2025-01-01'),
                  to_date('2025-12-31'),
                  interval 1 day
                          
                 )
             ) AS date
      """) 
     window_spec = Window.orderBy("date")

     dim_date = dim_date.select(
         F.row_number().over(window_spec).alias("date_key"),   # key فريد 1 → 365
         F.date_format("date", "dd-MM-yyyy").alias("full_date"),
         F.date_format("date", "EEEE").alias("day_name"),
         F.date_format("date", "MMMM").alias("month_name")
     )
         

     # تغير الداتا تايب بتاع الديت 
     dim_date = dim_date.withColumn(
          "full_date",
         F.to_date(F.col("full_date"),"dd-MM-yyyy")
         )

     #dim_date.show(10, truncate=False)
     return(dim_date)


#=====================================================
# witer in s3 
def write_dimention (df ,path):
        
    df.write \
          .mode("overwrite") \
          .option("header","true") \
          .csv(path)

#===================================================
#main def 
def main():
     
     spark = (
         SparkSession.builder
         .appName("dw")

         # ✅ Hadoop S3A jars 
         .config(
             "spark.jars.packages",
             "org.apache.hadoop:hadoop-aws:3.4.0,"
             "com.amazonaws:aws-java-sdk-bundle:1.12.262"
         )
            

         # MinIO
         .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
         .config("spark.hadoop.fs.s3a.path.style.access", "true")
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

         .getOrCreate()
    )

     # هنجيب اول داتا ونحطها في داتا فريم streets_data
     streets_data = spark.read.csv(
         "s3a://datalake/processed/streets_data",
         header=True,
         inferSchema=True
     )

     # هنجيب ثاني  داتا ونحطها في داتا فريم Motor_Vehicle_Collisions_Crashes
     Motor_Vehicle = spark.read.csv(
         "s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes",
         header=True,
         inferSchema=True
     )

     # هنجيب ثالث  داتا ونحطها في داتا فريم mongo_traffic
     complaint_311 = spark.read.csv(
         "s3a://datalake/processed/complaint_311",
         header=True,
         inferSchema=True
     )
     
     dimentions={
          "Dim_Borough":Dim_Borough(Motor_Vehicle),
          "Dim_Status":Dim_Status(complaint_311),
          "Dim_Traffic_Volume_Level":Dim_Traffic_Volume_Level(complaint_311),
          "Dim_Vehicle_Type":Dim_Vehicle_Type(Motor_Vehicle),
          "Dim_Weather":Dim_Weather(complaint_311),
          "Dim_Contributing":Dim_Contributing(Motor_Vehicle),
          "Dim_Street":Dim_Street(Motor_Vehicle,complaint_311,streets_data),
          "Dim_Complaint":Dim_Complaint(complaint_311),
          "Dim_Date":Dim_Date(spark)
     }
     for dim_name , dim_df in dimentions.items():
          write_dimention(dim_df,f"s3a://datalake/data_warehouse(dim)/{dim_name}")


if __name__ == "__main__":
     main()

26/01/15 03:36:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+------+
|status_key|status|
+----------+------+
|0         |closed|
|1         |open  |
+----------+------+



26/01/15 03:37:09 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:10 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:13 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:19 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/15 03:37:20 WA

task 6 
تكوين الشكل الفاكت تيبولز 
رفعهم علي س3

In [1]:
#task 6
from pyspark.sql.functions import monotonically_increasing_id , col
from pyspark.sql import SparkSession

#------------------------------------------
# create Fact_Collisions
def Fact_Collisions(Motor_Vehicle,Dim_Date,Dim_Borough,Dim_Vehicle_Type,Dim_Street,Dim_Contributing):
     
     # هنعمل جوين مع ال Dim_Date     
     fact_collisions = Motor_Vehicle.join(
         Dim_Date,
         Motor_Vehicle.crash_date == Dim_Date.full_date,
         "left"
      )
     #------------------------------------------------
     #هنعمل جوين مع Dim_Borough
     fact_collisions = fact_collisions.join(
         Dim_Borough,
         fact_collisions.borough == Dim_Borough.borough,
         "left"
     )

     #------------------------------------------------
     # هنعمل جوين مع Dim_Vehicle_Type
     fact_collisions= fact_collisions.join(
         Dim_Vehicle_Type,
         fact_collisions.vehicle_type==Dim_Vehicle_Type.vehicle_name,
         "left"
     )

     #--------------------------------------------------
     # هنعمل جوين مع Dim_Street
     fact_collisions= fact_collisions.join(
         Dim_Street,
         fact_collisions.street_name==Dim_Street.street_name,
         "left"
     )

     #--------------------------------------------------
     # هنعمل جوين مع Dim_Contributing
     fact_collisions= fact_collisions.join(
          Dim_Contributing,
         fact_collisions.contributing_factor==Dim_Contributing.contributing_factor,
        "left"
     )

     #--------------------------------------------------------
     # هنبدء بقا نختار الاعمده الي عوزينها بس عشان نكون الفاكت تيبول بتاعنا
     fact_collisions=fact_collisions.select(
         "collision_id",
         "date_key",
         "borough_id",
         "vehicle_type_id",
         "street_id",
         "factor_key",
         "crash_time",
         "persons_injured",
         "persons_killed"
     )
     #--------------------------------------------------------------------
     #  نعرض عينة للتأكد
     #fact_collisions.show(10)
     return(fact_collisions)

#------------------------------------------
#write in s3
def write_fact(df,path):
     df.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv(path)

#------------------------------------------
# create Fact_Complaint
def Fact_Complaint(complaint_311,Dim_Date,Dim_Borough,Dim_Status,Dim_Traffic_Volume_Level,Dim_Street,Dim_Weather,Dim_Complaint):
     
     # هنعمل جوين مع ال Dim_Date     
     fact_complaint = complaint_311.join(
         Dim_Date,
         complaint_311.created_date == Dim_Date.full_date,
         "left"
     )
     #------------------------------------------------
     #هنعمل جوين مع Dim_Borough
     fact_complaint = fact_complaint.join(
         Dim_Borough,
         fact_complaint.borough == Dim_Borough.borough,
         "left"
     )

     #------------------------------------------------
     # هنعمل جوين مع Dim_Status
     fact_complaint= fact_complaint.join(
         Dim_Status,
         fact_complaint.status==Dim_Status.status,
         "left"
     )

     #--------------------------------------------------
     # هنعمل جوين مع Dim_Traffic_Volume_Level
     fact_complaint= fact_complaint.join(
         Dim_Traffic_Volume_Level,
         fact_complaint.traffic_volume_level==Dim_Traffic_Volume_Level.volume_level,
         "left"
     )

     #--------------------------------------------------
     # هنعمل جوين مع Dim_Street
     fact_complaint= fact_complaint.join(
         Dim_Street,
         fact_complaint.street_name==Dim_Street.street_name,
         "left"
     )
     #--------------------------------------------------
     # هنعمل جوين مع Dim_Weather
     fact_complaint= fact_complaint.join(
         Dim_Weather,
         fact_complaint.weather_condition==Dim_Weather.weather_condition,
         "left"
     )
     #--------------------------------------------------
     # هنعمل جوين مع Dim_Complaint
     fact_complaint= fact_complaint.join(
         Dim_Complaint,
         fact_complaint.complaint_type==Dim_Complaint.complaint_type,
         "left"
     )


     #--------------------------------------------------------
     # هنبدء بقا نختار الاعمده الي عوزينها بس عشان نكون الفاكت تيبول بتاعنا
     fact_complaint=fact_complaint.select(
         "complaint_311_id",
         "date_key",
         "borough_id",
         "status_key",
         "street_id",
         "volume_level_key",
         "complaint_key",
         "weather_key",
         "severity",
         "pepole_injuries",
         "pepole_killed",
         "response_time_minutes",
         "vehicles_involved"
     )
     #--------------------------------------------------------------------
     
     return(fact_complaint)




def main():
     
     spark = (
         SparkSession.builder
         .appName("dw(facts)")

         # ✅ Hadoop S3A jars 
         .config(
             "spark.jars.packages",
             "org.apache.hadoop:hadoop-aws:3.4.0,"
             "com.amazonaws:aws-java-sdk-bundle:1.12.262"
         )
            

         # MinIO
         .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
         .config("spark.hadoop.fs.s3a.path.style.access", "true")
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

         .getOrCreate()
    )
    
     #===============================================
     # هنجيب اول داتا ونحطها في داتا فريم streets_data
     streets_data = spark.read.csv(
         "s3a://datalake/processed/streets_data",
         header=True,
         inferSchema=True
     )

     #===============================================
     # هنجيب ثاني  داتا ونحطها في داتا فريم Motor_Vehicle_Collisions_Crashes
     Motor_Vehicle = spark.read.csv(
         "s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes",
         header=True,
         inferSchema=True
     )

     #===============================================
     # هنجيب ثالث  داتا ونحطها في داتا فريم mongo_traffic
     complaint_311 = spark.read.csv(
         "s3a://datalake/processed/complaint_311",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 1-git Dim_Borough
     Dim_Borough = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Borough",
         header=True,
         inferSchema=True
     )

     #===============================================
     # 2-git Dim_Complaint
     Dim_Complaint = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Complaint",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 3- git Dim_Contributing
     Dim_Contributing = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Contributing",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 4-git Dim_Status
     Dim_Status = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Status",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 5-git Dim_Date
     Dim_Date = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Date",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 6-git Dim_Street
     Dim_Street = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Street",
         header=True,
         inferSchema=True
     )
     #===============================================
     # 7-git Dim_Traffic_Volume_Level
     Dim_Traffic_Volume_Level = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Traffic_Volume_Level",
         header=True,
         inferSchema=True
     )
       #===============================================
     # git Dim_Weather
     Dim_Weather = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Weather",
         header=True,
         inferSchema=True
     )
       #===============================================
     # git Dim_Vehicle_Type
     Dim_Vehicle_Type = spark.read.csv(
         "s3a://datalake/data_warehouse(dim)/Dim_Vehicle_Type",
         header=True,
         inferSchema=True
     )






     facts={
          "Fact_Collisions":Fact_Collisions(Motor_Vehicle,Dim_Date,Dim_Borough,Dim_Vehicle_Type,Dim_Street,Dim_Contributing),
          "Fact_Complaint":Fact_Complaint(complaint_311,Dim_Date,Dim_Borough,Dim_Status,Dim_Traffic_Volume_Level,Dim_Street,Dim_Weather,Dim_Complaint),
         
     }
     for fact_name , fact_df in facts.items():
          write_fact(fact_df,f"s3a://datalake/data_warehouse(fact)/{fact_name}")


if __name__ == "__main__":
     main()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/16 00:20:41 WARN Utils: Your hostname, refat, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
26/01/16 00:20:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahmed-refat/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ahmed-refat/.ivy2.5.2/cache
The jars for the packages stored in: /home/ahmed-refat/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f8ad6a6-7880-465e-955f-f460299d70ee;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.0 in central
	found software.amazon.awssdk#bundle;2.23.19 in central
	found org.wildfly.openssl#wildfly-open

task 7
load data to data warehouse in snowflake 

In [5]:
from pyspark.sql import SparkSession


spark = (
     SparkSession.builder
     .appName("load")

      # ✅ Hadoop S3A jars 
     .config(
         "spark.jars.packages",
         "org.apache.hadoop:hadoop-aws:3.4.0,"
         "com.amazonaws:aws-java-sdk-bundle:1.12.262"
     )
            

     # MinIO
     .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
     .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
     .config("spark.hadoop.fs.s3a.path.style.access", "true")
     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

     .getOrCreate()
)

    
#===============================================
# 1-git Dim_Borough
Dim_Borough = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Borough",
     header=True,
     inferSchema=True
)

#===============================================
# 2-git Dim_Complaint
Dim_Complaint = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Complaint",
     header=True,
     inferSchema=True
)
    
#===============================================
# 3- git Dim_Contributing
Dim_Contributing = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Contributing",
     header=True,
     inferSchema=True
)
#===============================================
# 4-git Dim_Status
Dim_Status = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Status",
     header=True,
     inferSchema=True
)
#===============================================
# 5-git Dim_Date
Dim_Date = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Date",
     header=True,
     inferSchema=True
)
#===============================================
# 6-git Dim_Street
Dim_Street = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Street",
     header=True,
     inferSchema=True
)
#===============================================
# 7-git Dim_Traffic_Volume_Level
Dim_Traffic_Volume_Level = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Traffic_Volume_Level",
     header=True,
     inferSchema=True
)
#===============================================
# 8-git Dim_Weather
Dim_Weather = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Weather",
     header=True,
     inferSchema=True
)
#===============================================
# 9-git Dim_Vehicle_Type
Dim_Vehicle_Type = spark.read.csv(
     "s3a://datalake/data_warehouse(dim)/Dim_Vehicle_Type",
     header=True,
     inferSchema=True
)

#========================================================================================================
# 1-git Fact_Complaint
Fact_Complaint = spark.read.csv(
     "s3a://datalake/data_warehouse(fact)/Fact_Complaint",
     header=True,
     inferSchema=True
)

#===============================================
# 2-git Fact_Collisions
Fact_Collisions = spark.read.csv(
     "s3a://datalake/data_warehouse(fact)/Fact_Collisions",
     header=True,
     inferSchema=True
)

Dim_Vehicle_Type.printSchema()


root
 |-- vehicle_type_id: integer (nullable = true)
 |-- vehicle_name: string (nullable = true)



In [13]:
Fact_Complaint.printSchema()

root
 |-- complaint_311_id: string (nullable = true)
 |-- date_key: integer (nullable = true)
 |-- borough_id: integer (nullable = true)
 |-- status_key: integer (nullable = true)
 |-- street_id: integer (nullable = true)
 |-- volume_level_key: integer (nullable = true)
 |-- complaint_key: integer (nullable = true)
 |-- weather_key: integer (nullable = true)
 |-- severity: string (nullable = true)
 |-- pepole_injuries: integer (nullable = true)
 |-- pepole_killed: integer (nullable = true)
 |-- response_time_minutes: integer (nullable = true)
 |-- vehicles_involved: integer (nullable = true)

